In [37]:
import pandas as pd
dating = pd.read_csv("profiles.csv")

In [38]:
# basic data cleanup
# income
dating.loc[dating["income"] == -1, "income"] = pd.NA

# Let's create ML algorithm that will predict drug use on empty fields

prepare data some more for drug investigation

In [39]:
# create some suspicios words list that may relate to drug use to filter from essays
drug_words = ["party", "fun", "hang out", "night life", "festival", "good time", "drink"]
essay_cols = [col for col in dating.columns if col.startswith("essay")]
for word in drug_words:
    dating[word + "_count"] = dating[essay_cols].apply(
        lambda row: sum(str(cell).lower().count(word) for cell in row if pd.notna(cell)),
        axis=1
    )
    
dating = dating.drop(columns=essay_cols)

In [67]:
drugs_values = dating["drugs"].value_counts()
dating_drugs_filled = dating.dropna(subset=["drugs"])
dating_drugs_missing = dating[dating["drugs"].isna()]
features = dating_drugs_filled.drop(columns=["drugs"])
labels = dating_drugs_filled["drugs"]
features["education"][features["education"].str.contains("graduated", na=False)].value_counts()

education
graduated from college/university    17706
graduated from masters program        7022
graduated from two-year college       1303
graduated from high school            1223
graduated from ph.d program           1028
graduated from law school              804
graduated from space camp              498
graduated from med school              370
Name: count, dtype: int64

In [68]:
# prepare features for ML
features = pd.get_dummies(features, columns=["body_type", "drinks"], dummy_na=False)


# bucket education column
def bucket_education(val):
    if pd.isna(val):
        return pd.NA
    val = str(val).lower()
    is_working_on = "working on" in val
    has_college_or_uni = any([exp in val for exp in ["college", "university", "masters program"]])
    has_phd_program = "ph.d program" in val
    has_graduated = "graduated" in val
    has_high_school = any(exp in val for exp in ["high school", "law school", "med school", "space camp"])
    has_dropped_out = "dropped out" in val

    if has_phd_program and not is_working_on and not has_dropped_out:
        return "Ph.D"

    if has_graduated and (has_college_or_uni or has_phd_program) or has_phd_program:
        return "graduated"

    if (has_college_or_uni and not is_working_on and not has_dropped_out):
        return "graduated"

    if (has_high_school and not is_working_on and not has_dropped_out) or (
        (is_working_on or has_dropped_out) and has_college_or_uni
    ):
        return "high_school"

    return "lower"


features["education_bucket"] = features["education"].apply(bucket_education)
features = pd.get_dummies(features, columns=["education_bucket"], dummy_na=False)
# features = features.drop(columns=["education"])

In [69]:
education_unique = dating["education"].dropna().unique()
education_buckets = {edu: bucket_education(edu) for edu in education_unique}
education_buckets

{'working on college/university': 'high_school',
 'working on space camp': 'lower',
 'graduated from masters program': 'graduated',
 'graduated from college/university': 'graduated',
 'working on two-year college': 'high_school',
 'graduated from high school': 'high_school',
 'working on masters program': 'high_school',
 'graduated from space camp': 'high_school',
 'college/university': 'graduated',
 'dropped out of space camp': 'lower',
 'graduated from ph.d program': 'Ph.D',
 'graduated from law school': 'high_school',
 'working on ph.d program': 'graduated',
 'two-year college': 'graduated',
 'graduated from two-year college': 'graduated',
 'working on med school': 'lower',
 'dropped out of college/university': 'high_school',
 'space camp': 'high_school',
 'graduated from med school': 'high_school',
 'dropped out of high school': 'lower',
 'working on high school': 'lower',
 'masters program': 'graduated',
 'dropped out of ph.d program': 'graduated',
 'dropped out of two-year colleg

let's go with decision tree model

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=.2, random_state=21)
dtree = DecisionTreeClassifier(max_depth=6, random_state=10)
dtree.fit(x_train, y_train)

ValueError: could not convert string to float: 'mostly anything'